In [1]:

import os
import sys
import pinecone

currentdir = os.getcwd()

parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

import components.pinecone_langchain as plc


import scrapeandindex.sparsevectors as sv
import scrapeandindex.query as query_sql


/Users/juliev/miniforge3/envs/metrics/lib/python3.11/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


running on cpu


In [2]:
pinecone_key = os.getenv('PINECONE_API_KEY')
pinecone_env = os.getenv('PINECONE_ENV')
pinecone.init(api_key=pinecone_key, environment=pinecone_env)

In [3]:
pinecone.list_indexes()

['pinehack']

In [4]:
info = pinecone.Index('pinehack')

In [5]:
info

In [6]:
info.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'langchaindocs': {'vector_count': 1273}},
 'total_vector_count': 1273}

In [7]:
pilang = plc.LangChainPineconeClient()

In [8]:
## This is the only function needed by the frontend
pilang.ask_with_context("Langchain is a tool for categorizing text into different topics")

'As an AI language model, I cannot verify the accuracy of the information provided in the source. However, based on the information provided, the text appears to be up to date and accurate. The source describes LangChain as a framework for developing applications powered by language models, which enables applications that are data-aware and agentic. It also highlights the main value propositions of LangChain, including components and off-the-shelf chains, which make it easy to get started and customize existing chains or build new ones. Finally, the source confirms that LangChain is a tool for categorizing text into different topics. Source: Introduction LangChain is a framework for developing applications powered by language models. It enables applications that are: Data-aware: connect a language model to other sources of data Agentic: allow a language model to interact with its environment The main value props of LangChain are: Components: abstractions for working with language model

In [ ]:
pilang.get_relevant_text("How do I ask questions over documents using LangChain?", "LangChain")

AIMessage(content='LangChain is the subject of the document.', additional_kwargs={})

In [ ]:
pilang.get_relevant_pinecone_data("How do I ask questions over documents using LangChain?")

'Document Question Answering\u200b Question answering involves fetching multiple documents, and then asking a question of them.\nThe LLM response will contain the answer to your question, based on the content of the documents. The recommended way to get started using a question answering chain is: from langchain . chains . question_answering import load_qa_chain chain = load_qa_chain ( llm , chain_type = "stuff" ) chain . run ( input_documents = docs , question = query ) The following resources exist: Question Answering Notebook: A notebook walking through how to accomplish this task. VectorDB Question Answering Notebook: A notebook walking through how to do question answering over a vector database. This can often be useful for when you have a LOT of documents, and you don\'t want to pass them all to the LLM, but rather first want to do some semantic search over embeddings.'

In [ ]:
pilang.index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'langchaindocs': {'vector_count': 1273}},
 'total_vector_count': 1273}

In [ ]:
embedded_query = pilang.embed.embed_query("How do I ask questions over documents using LangChain?")

In [ ]:
result = pilang.index.query(
namespace='langchaindocs',
top_k=1,
vector=embedded_query,
sparse_vector = sv.get_sparse_vector("How do I ask questions over documents using LangChain?"))

In [ ]:
match_id = result['matches'][0]['id']

In [ ]:
result

{'matches': [{'id': '875899727194390529', 'score': 25.1728687, 'values': []}],
 'namespace': 'langchaindocs'}

In [2]:
import psycopg2
DATABASE_URL = os.environ['DATABASE_URL']
conn = psycopg2.connect(DATABASE_URL)
cur = conn.cursor()

In [5]:
entry = query_sql.get_heading_by_rowid('875899727194390529', cur)

In [6]:
entry

('9',
 'Document Question Answering\u200b',
 '{}',
 '{1636,1637,1638,1639,1640,1641,1642,1643,1644,1645,1646,1647,1648,1649,1650,1651,1652,1653,1654,1655,1656,1657,1658,1659,1660,1661,1662,1663,1664,1665,1666,1667,1668,1669,1670}',
 True,
 'Document Question Answering\u200b Question answering involves fetching multiple documents, and then asking a question of them.\nThe LLM response will contain the answer to your question, based on the content of the documents. The recommended way to get started using a question answering chain is: from langchain . chains . question_answering import load_qa_chain chain = load_qa_chain ( llm , chain_type = "stuff" ) chain . run ( input_documents = docs , question = query ) The following resources exist: Question Answering Notebook: A notebook walking through how to accomplish this task. VectorDB Question Answering Notebook: A notebook walking through how to do question answering over a vector database. This can often be useful for when you have a LOT o

In [7]:
entry[5]

'Document Question Answering\u200b Question answering involves fetching multiple documents, and then asking a question of them.\nThe LLM response will contain the answer to your question, based on the content of the documents. The recommended way to get started using a question answering chain is: from langchain . chains . question_answering import load_qa_chain chain = load_qa_chain ( llm , chain_type = "stuff" ) chain . run ( input_documents = docs , question = query ) The following resources exist: Question Answering Notebook: A notebook walking through how to accomplish this task. VectorDB Question Answering Notebook: A notebook walking through how to do question answering over a vector database. This can often be useful for when you have a LOT of documents, and you don\'t want to pass them all to the LLM, but rather first want to do some semantic search over embeddings.'

In [8]:
selection

NameError: name 'selection' is not defined

In [9]:

from scrapeandindex.query import select_row_by_index
# get the heading in the 5th row of the headings table
heading = select_row_by_index(500, 'headings', '*', conn)
# print(heading)



In [10]:
heading

('66',
 'Databricks',
 '{502,503,507}',
 '{5581}',
 True,
 'Databricks This notebook covers how to connect to the Databricks runtimes and Databricks SQL using the SQLDatabase wrapper of LangChain.\nIt is broken into 3 parts: installation and setup, connecting to Databricks, and examples.')

In [11]:
# get the url at the urlid in the headings table
url = select_row_by_index(heading[0], 'urls', '*', conn)
# print(url)

In [12]:
url

('https://python.langchain.com/docs/ecosystem/integrations/databricks/',
 True,
 datetime.datetime(2023, 6, 21, 15, 6, 59, 97449))

In [3]:
query_sql.get_url_by_headingid('875899727194390529', conn)

('https://python.langchain.com/docs/use_cases/question_answering/',
 True,
 datetime.datetime(2023, 6, 21, 14, 56, 23, 132373))